# Packages

In [ ]:
# Matplotlib setup
from matplotlib_setup import mpl_setup

# Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager
import matplotlib.gridspec as gridspec
import seaborn as sns

# Ancillary functions

In [ ]:
# lighten/darken colour
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

# Setup

In [ ]:
# directories
wd = '/Users/lneumann/Documents/'
database_dir = wd + 'Products/ALMOND/dense_gas_letter/database/'
tab_dir =  wd + 'Products/ALMOND/dense_gas_letter/tables/'
plot_dir = wd + 'Products/ALMOND/dense_gas_letter/plots/appendix/'

# load dense gas literature
fname_table = database_dir + 'gao_solomon_literature_compilation_2024.csv'
table = pd.read_csv(fname_table)

# conversions
tir2sfr = 1.48e-10  # [Msun/yr] Murphy+11
alpha_hcn = 15  # [M_Sun/pc^2/(K km/s)] (Schinnerer & Leroy 2024)

# SFE median across literature compilation (2024)
fname_gs_table = tab_dir + 'GS_relation_per_regime.csv'
gs_table = pd.read_csv(fname_gs_table)
sfe_median = np.array(gs_table[gs_table['data']=='combined']['SFR/HCN (median)'])[0]
sfe_scatter = np.array(gs_table[gs_table['data']=='combined']['scatter'])[0]

# ALMOND + EMPIRE
color_almond = lighten_color('tab:blue', amount=0.7)
color_empire = lighten_color('tab:red', amount=0.7)

# sub-samples
table_clouds = table[table['Type'] == 'clouds and clumps']
table_resolved = table[table['Type'] == 'parts of galaxies']
table_galaxies = table[table['Type'] == 'entire galaxies']
color_clouds = 'tab:green'
color_resolved = 'tab:olive'
color_galaxies = 'tab:pink'
color_mw_cmz = 'orange'

# references
refs_clouds = ['Chin97','Chin98','Brouillet05','Wu10','Lada12','Buchbender13','Evans14','Stephens16','Braine17']
colors_clouds = ['xkcd:spring green', 'xkcd:pale yellow', 'xkcd:pale pink', 'xkcd:powder blue', 'xkcd:greyish green', 'xkcd:pumpkin', 'xkcd:light red', 'xkcd:ochre', 'xkcd:avocado']
markers_clouds = ['s'] * len(refs_clouds)

refs_resolved = ['Kepley14', 'Usero15', 'Bigiel15', 'Chen17', 'Gallagher18', 'Jiménez-Donaire19', 'Querejeta19', 'Bešlić21', 'Eibensteiner22', 'Sánchez-García22', 'Neumann23', 'Stuber23', 'Bešlić24', 'Neumann24']
colors_resolved = ['tab:green', 'tab:purple', 'tab:brown', 'limegreen', 'tab:pink', color_empire, 'tab:orange', 'tab:gray', 'tab:cyan', 'tan', color_almond, 'tab:olive', 'xkcd:dark pink', 'm']
markers_resolved = ['o'] * len(refs_resolved)

refs_galaxies = ['Gao04','Gao07','Krips08','Gracia-Carpio08','Juneau09','Crocker12','Garcia-Burillo12','Privon15','Rybak22']
colors_galaxies = ['xkcd:sky blue', 'xkcd:teal', 'xkcd:burnt orange', 'xkcd:mauve', 'xkcd:mustard', 'xkcd:coral', 'xkcd:light green', 'xkcd:dull blue', 'xkcd:aubergine']
markers_galaxies = ['D'] * len(refs_galaxies)

# combine refs
refs_tot = refs_clouds + refs_resolved + refs_galaxies
colors_tot = colors_clouds + colors_resolved + colors_galaxies
markers_tot = markers_clouds + markers_resolved + markers_galaxies

# sort by publication date
refs_tot_yr = [np.array(table['Publication date'][table['Name']==ref])[0] for ref in refs_tot]
refs_tot_yr = [float(ref.split('/')[1]) + float(ref.split('/')[0])/12 for ref in refs_tot_yr]
refs_tot_yr, refs_tot, colors_tot, markers_tot = zip(*sorted(zip(refs_tot_yr, refs_tot, colors_tot, markers_tot)))


# secondary y-axis for SFR to TIR
def sfr_to_tir(sfr):
    return sfr / tir2sfr
def tir_to_sfr(tir):
    return tir * tir2sfr

# secondary x-axis for HCN to Mdense
def hcn_to_mdense(hcn):
    return hcn * alpha_hcn
def mdense_to_hcn(mdense):
    return mdense / alpha_hcn

# Plotting

In [ ]:
# toogle save figure
savepng = False
savepdf = True

# matplotlib setup
mpl_setup(figtype='paper-1/1', ms=3, aspect=1.3)

# create figure
fig = plt.figure()
fig.subplots_adjust(hspace=0, wspace=0)

# handles for legend
handles_clouds = []
handles_resolved = []
handles_galaxies = []
handles_clouds_grey = []
handles_resolved_grey = []
handles_galaxies_grey = []
handles_dense = []

# iterate over references
for ref, color, marker in zip(refs_tot, colors_tot, markers_tot):

    # loop index
    i = refs_tot.index(ref)
    
    # subplot
    nrows = 8
    ncols = 4
    ax = fig.add_subplot(nrows, ncols, i+1)

    # get data
    ids = (table['Name']==ref)
    x = table['L_HCN [K.km/s.pc2]'][ids]
    y = table['SFR [Msun/yr]'][ids]

    # plot data
    label = ref[:-2] + ' et al. 20' + ref[-2:] if int(ref[-2:]) < 50 else ref[:-2] + ' et al. 19' + ref[-2:]
    handle, = ax.plot(x, y, marker=marker, mfc=color, mec='k', mew=0.5, ls='none', label=label, zorder=3)


    # plot all literature in grey
    for ref, color, marker in zip(refs_tot, colors_tot, markers_tot):
    
        # get data
        ids = (table['Name']==ref)
        x = table['L_HCN [K.km/s.pc2]'][ids]
        y = table['SFR [Msun/yr]'][ids]
    
        # plot data
        label = ref[:-2] + ' et al. 20' + ref[-2:] if int(ref[-2:]) < 50 else ref[:-2] + ' et al. 19' + ref[-2:]
        handle_grey, = ax.plot(x, y, marker=marker, c=lighten_color('grey', amount=0.5), mew=0.5, ls='none', label=label)

    # append handle to sample list
    if marker == 's':
        handles_clouds.append(handle_grey)
    elif marker == 'o':
        handles_resolved.append(handle_grey)
    elif marker == 'D':
        handles_galaxies.append(handle_grey)


    ################
    
    # plot median SFE line
    x_values = np.logspace(-2,12,100)
    # SFR vs HCN
    y_values = x_values*10**sfe_median 
    ax.plot(x_values, y_values, c='k')
    ax.plot(x_values, y_values*10**sfe_scatter, c='k', ls='dashed')
    ax.plot(x_values, y_values/10**sfe_scatter, c='k', ls='dashed')
    ax.fill_between(x_values, y_values*10**sfe_scatter, y_values/10**sfe_scatter, color='grey', alpha=0.3, lw=0)


    ################

    
    # general settings
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlim(1e-1, 3e11)
    ax.set_ylim(1e-1*10**sfe_median, 3e11*10**sfe_median)
    ax.set_xticks(np.logspace(0,10,6))
    ax.set_yticks(np.logspace(-6,4,6))

    if i % ncols == (ncols-1):
        # secondary y-axis for L_IR
        secyax = ax.secondary_yaxis('right', functions=(sfr_to_tir, tir_to_sfr)) 
        ax.tick_params(which='both', right=False)
        secyax.set_yticks(np.logspace(3,13,6))
    
    if i % ncols != 0:
        ax.tick_params(labelleft=False)

    if i < ncols:
        # secondary x-axis for M_dense
        secxax = ax.secondary_xaxis('top', functions=(hcn_to_mdense, mdense_to_hcn)) 
        ax.tick_params(which='both', top=False)
        secxax.set_xticks(np.logspace(2,12,6))
        
    if i < (ncols*(nrows-1)):
        ax.tick_params(labelbottom=False)

    
    # ax.tick_params(labelbottom=False)
    

    ################

    # legend
    mpl.rcParams['text.usetex'] = False
    font = font_manager.FontProperties(family='Arial', style='normal', size=6)
    legend1 = ax.legend(handles=[handle], loc='upper left', prop=font)
    mpl.rcParams['text.usetex'] = True 
    
    #####################################


    
    ## PLOT MW CMZ
    # x = table['L_HCN [K.km/s.pc2]'][table['Name']=='Henshaw23']
    # y = table['SFR [Msun/yr]'][table['Name']=='Henshaw23']
    # handle_cmz, = ax1.plot(x, y, marker='*', ms=10, mfc=color_mw_cmz, mec='k', mew=0.5, ls='none', label='Jones et al. 2012 &\n Barnes et al. 2017', zorder=5)
    # handle_cmz_grey, = ax2.plot(x, y/x, marker='*', ms=10, mfc=lighten_color('grey', amount=0.5), mec='grey', mew=0.5, ls='none', label='Jones et al. 2012 &\n Barnes et al. 2017', zorder=5)
    
    ################

    
# axis labels
fig.text(0.5, 0.07, '$L_\mathrm{HCN}$ [K$\,$km$\,\mathrm{s}^{-1}\,\mathrm{pc}^2$]', ha='center', va='bottom')
fig.text(0.5, 0.925, '$M_\mathrm{dense}$ [$\mathrm{M}_{\odot}$]', ha='center', va='top')
fig.text(0.05, 0.5, 'SFR [$\mathrm{M}_{\odot}\,\mathrm{yr}^{-1}$]', ha='left', va='center', rotation=90)
fig.text(0.965, 0.5, '$L_\mathrm{IR}^\mathrm{equiv}$ [L$_{\odot}$]', ha='right', va='center', rotation=90)

# save plot
savepath = plot_dir + 'GS_relation_individual_studies'
if savepng:
    plt.savefig(savepath + '.png')
if savepdf:
    plt.savefig(savepath + '.pdf')

plt.show()